In [1]:
import os

In [2]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW'

In [5]:
import pandas as pd 

In [7]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [9]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [10]:
data.shape

(1599, 12)

In [11]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_dir: Path 
    all_schema: dict 

In [14]:
from mlProject.constants import * 
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH, 
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config_filepath = read_yaml(config_filepath)
        self.params_file_path = read_yaml(params_file_path)
        self.schema_file_path = read_yaml(schema_file_path)
        
        create_directories([self.config_filepath.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config_filepath.data_validation
        schema = self.schema_file_path.COLUMNS 
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_dir = config.unzip_dir,
            all_schema = schema,
            )
        return data_validation_config

In [27]:
import os
from mlProject import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_data(self) -> bool:
        try:
            validation_status = None 
            
            data = pd.read_csv(self.config.unzip_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False 
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status for {col} in the data : {validation_status} ")
                        break
                else:
                    validation_status = True 
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status : {validation_status}")
        except Exception as e:
            logger.info(f"Error while validating as {e.message}")
            logger.exception(e)
            raise e
        
        return validation_status

In [28]:
try:
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_data()
except Exception as e:
    raise e 

[2023-12-20 22:37:02,590: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-20 22:37:02,591: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-20 22:37:02,593: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-20 22:37:02,595: INFO: common: created directory at: artifacts]
[2023-12-20 22:37:02,596: INFO: common: created directory at: artifacts/data_validation]
